In [1]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, create_optimizer
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import shutil

E0000 00:00:1721812856.412692      77 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0724 09:20:56.420779075      77 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0724 09:20:56.420792433      77 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0724 09:20:56.420795598      77 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0724 09:20:56.420797968      77 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0724 09:20:56.420800678      77 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Ініціалізація TPU
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1721812891.737301      77 service.cc:145] XLA service 0x577cafa3e2a0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721812891.737359      77 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1721812891.737364      77 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1721812891.737367      77 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1721812891.737370      77 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1721812891.737372      77 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1721812891.737375      77 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1721812891.737378      77 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1721812891.737384      77 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


In [3]:
# Завантаження датасету
file_path = '/kaggle/input/toxic-dataset/train_data_5000on5000.csv'  # Update the path to your dataset
data = pd.read_csv(file_path)

# Попередня обробка датасету
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['comment_text'].tolist(), 
    data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist(), 
    test_size=0.1
)

# Токенізатор
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Функція токенізації текстів
def tokenize_texts(texts, labels, tokenizer, max_length=512):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])

    return (
        np.array(input_ids),
        np.array(attention_masks),
        np.array(labels)
    )

# Токенізація датасетів
train_input_ids, train_attention_masks, train_labels = tokenize_texts(train_texts, train_labels, tokenizer)
val_input_ids, val_attention_masks, val_labels = tokenize_texts(val_texts, val_labels, tokenizer)

# Функція створення TensorFlow датасетів
def create_tf_dataset(input_ids, attention_masks, labels, batch_size=32):
    dataset = Dataset.from_tensor_slices(({'input_ids': input_ids, 'attention_mask': attention_masks}, labels))
    dataset = dataset.shuffle(buffer_size=len(input_ids)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

BATCH_SIZE = 32

train_dataset = create_tf_dataset(train_input_ids, train_attention_masks, train_labels, BATCH_SIZE)
val_dataset = create_tf_dataset(val_input_ids, val_attention_masks, val_labels, BATCH_SIZE)

In [4]:
with strategy.scope():
    # Завантаження базової моделі BERT для класифікації
    base_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

    # Вхідні дані
    input_ids = tf.keras.Input(shape=(512,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.Input(shape=(512,), dtype=tf.int32, name='attention_mask')

    # Проходження через базову модель BERT
    outputs = base_model.bert(input_ids, attention_mask=attention_mask)
    pooler_output = outputs[1]  # Використання pooler_output

    # Додавання нових шарів
    x = tf.keras.layers.Dropout(0.3)(pooler_output)
    x = tf.keras.layers.Dense(124, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    output = tf.keras.layers.Dense(6, activation='sigmoid')(x)

    # Створення нової моделі
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

    # Параметри тренування
    EPOCHS = 3
    steps_per_epoch = len(train_texts) // BATCH_SIZE
    num_train_steps = steps_per_epoch * EPOCHS
    num_warmup_steps = int(num_train_steps * 0.1)

    # Оптимізатор та функція втрат
    optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=num_warmup_steps, num_train_steps=num_train_steps)
    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

    # Компіляція моделі
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

    # Виведення архітектури моделі
    model.summary()

I0000 00:00:1721812914.460717      77 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


All PyTorch model weights were used when initializing TFBertForSequenceClassification.



Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model"


__________________________________________________________________________________________________


...

 dense_1 (Dense)             (None, 6)                    750       ['dropout_39[0][0]']          


Total params: 109578346 (418.01 MB)


Trainable params: 109578346 (418.01 MB)


Non-trainable params: 0 (0.00 Byte)


__________________________________________________________________________________________________


In [5]:
# Suppress AutoGraph warnings
@tf.autograph.experimental.do_not_convert
def train_model():
    with strategy.scope():
        # Training
        history = model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset)

        # Evaluate the model
        val_loss, val_acc = model.evaluate(val_dataset)
        print(f'Validation loss {val_loss} accuracy {val_acc}')



train_model()

Epoch 1/3


2024-07-24 09:23:19.505280: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


I0000 00:00:1721813002.418358     904 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(9b5113198618fc44:0:0), session_name()


I0000 00:00:1721813037.958453     904 tpu_compile_op_common.cc:245] Compilation of 9b5113198618fc44:0:0 with session name  took 35.540043176s and succeeded
I0000 00:00:1721813038.046817     904 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(9b5113198618fc44:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_train_function_1682034646335977722", property.function_library_fingerprint = 16317065752027948229, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "4,512,;4,512,;4,6,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1721813038.046902     904 tpu_compilation_cache_interface.cc:541] After adding entry for key 9b511

  1/282 [..............................] - ETA: 7:34:17 - loss: 0.7264 - accuracy: 0.2188

...

27/32 [========================>.....] - ETA: 0s - loss: 0.1475 - accuracy: 0.7963

29/32 [==========================>...] - ETA: 0s - loss: 0.1469 - accuracy: 0.7931

31/32 [============================>.] - ETA: 0s - loss: 0.1454 - accuracy: 0.7944

32/32 [==============================] - 2s 35ms/step - loss: 0.1448 - accuracy: 0.7940


Validation loss 0.14479492604732513 accuracy 0.7940000295639038


In [6]:
# Function to predict on a new comment
def predict_comment(comment, model, tokenizer):
    encoding = tokenizer.encode_plus(
        comment,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='tf',
        truncation=True
    )
    
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    
    preds = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask})[0]
    
    if preds.shape[0] == 1:
        preds = preds[0]  # Get the first element if it's a batch

    return {
        'toxic': preds[0],
        'severe_toxic': preds[1],
        'obscene': preds[2],
        'threat': preds[3],
        'insult': preds[4],
        'identity_hate': preds[5]
    }


In [7]:

model_save_path = "/kaggle/working/model_save"
model.save(model_save_path)
tokenizer.save_pretrained(model_save_path)


INFO:tensorflow:Assets written to: /kaggle/working/model_save/assets


INFO:tensorflow:Assets written to: /kaggle/working/model_save/assets


('/kaggle/working/model_save/tokenizer_config.json',
 '/kaggle/working/model_save/special_tokens_map.json',
 '/kaggle/working/model_save/vocab.txt',
 '/kaggle/working/model_save/added_tokens.json')

In [8]:
# Example usage
comment = "many think that he is a piece of shit and behaves like a bitch. but he's actually a nice guy"
prediction = predict_comment(comment, model, tokenizer)
print(prediction)

2024-07-24 09:27:41.371352: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


I0000 00:00:1721813262.021917     858 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(fed23d922af8b9c0:0:0), session_name()


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 14s 14s/step


{'toxic': 0.9706297, 'severe_toxic': 0.046682984, 'obscene': 0.9444723, 'threat': 0.022489816, 'insult': 0.88517576, 'identity_hate': 0.02267456}


I0000 00:00:1721813265.540877     858 tpu_compile_op_common.cc:245] Compilation of fed23d922af8b9c0:0:0 with session name  took 3.518909406s and succeeded
I0000 00:00:1721813265.562532     858 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(fed23d922af8b9c0:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_predict_function_10348529704568084209", property.function_library_fingerprint = 6345457749237655605, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "2,512,;2,512,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1721813265.562573     858 tpu_compilation_cache_interface.cc:541] After adding entry for key fed23d922

In [9]:
from sklearn.metrics import classification_report
# Generate predictions for the validation set and compute classification report

def evaluate_model(model, val_dataset, val_labels):
    # Отримання прогнозів для валідаційного набору
    val_predictions = model.predict(val_dataset)
    
    # Перетворення прогнозів у формат NumPy
    predictions = np.array(val_predictions).squeeze()

    # Бінаризація прогнозів
    predictions = np.where(predictions > 0.5, 1, 0)

    # Перетворення міток у формат NumPy
    true_labels = np.array(val_labels)

    # Виведення класифікаційного звіту
    report = classification_report(true_labels, predictions, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], zero_division=0)
    print(report)

# Виклик функції для оцінки моделі
evaluate_model(model, val_dataset, val_labels)

I0000 00:00:1721813276.831419     874 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(f7f1f2214d51538d:0:0), session_name()


 1/32 [..............................] - ETA: 7:43

 3/32 [=>............................] - ETA: 1s  

I0000 00:00:1721813280.675795     874 tpu_compile_op_common.cc:245] Compilation of f7f1f2214d51538d:0:0 with session name  took 3.844333843s and succeeded
I0000 00:00:1721813280.702118     874 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(f7f1f2214d51538d:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_predict_function_5342474885495931223", property.function_library_fingerprint = 38371434735699244, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "4,512,;4,512,;4,6,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1721813280.702152     874 tpu_compilation_cache_interface.cc:541] After adding entry for key f7f1f22

 5/32 [===>..........................] - ETA: 0s

...

I0000 00:00:1721813281.849393     889 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(763938e70b13ace7:0:0), session_name()


32/32 [==============================] - ETA: 0s

32/32 [==============================] - 20s 150ms/step


               precision    recall  f1-score   support

        toxic       0.48      0.51      0.49       466
 severe_toxic       0.11      0.04      0.06        48
      obscene       0.25      0.26      0.26       282
       threat       0.00      0.00      0.00        13
       insult       0.23      0.25      0.24       242
identity_hate       0.00      0.00      0.00        47

    micro avg       0.34      0.34      0.34      1098
    macro avg       0.18      0.18      0.18      1098
 weighted avg       0.32      0.34      0.33      1098
  samples avg       0.19      0.19      0.17      1098



I0000 00:00:1721813285.342025     889 tpu_compile_op_common.cc:245] Compilation of 763938e70b13ace7:0:0 with session name  took 3.492595179s and succeeded
I0000 00:00:1721813285.368304     889 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(763938e70b13ace7:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_predict_function_5342474885495931223", property.function_library_fingerprint = 38371434735699244, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "2,512,;2,512,;2,6,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1721813285.368337     889 tpu_compilation_cache_interface.cc:541] After adding entry for key 763938e